In [31]:
z = nib.load(atlas_path)
label_axis = z.header.get_axis(0)
print(label_axis.label)
for key, (label_name, rgba) in label_axis.label.items():
    print(key, label_name, rgba)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[{0: ('???', (1.0, 1.0, 1.0, 0.0)), 1: ('17Networks_LH_VisCent_ExStr_1', (0.470588, 0.0705882, 0.513726, 1.0)), 2: ('17Networks_LH_VisCent_ExStr_2', (0.470588, 0.0705882, 0.517647, 1.0)), 3: ('17Networks_LH_VisCent_ExStr_3', (0.470588, 0.0705882, 0.521569, 1.0)), 4: ('17Networks_LH_VisCent_ExStr_4', (0.470588, 0.0705882, 0.52549, 1.0)), 5: ('17Networks_LH_VisCent_ExStr_5', (0.470588, 0.0705882, 0.533333, 1.0)), 6: ('17Networks_LH_VisCent_ExStr_6', (0.470588, 0.0705882, 0.537255, 1.0)), 7: ('17Networks_LH_VisCent_Striate_1', (0.470588, 0.0705882, 0.541176, 1.0)), 8: ('17Networks_LH_VisCent_ExStr_7', (0.470588, 0.0705882, 0.545098, 1.0)), 9: ('17Networks_LH_VisCent_ExStr_8', (0.470588, 0.0705882, 0.54902, 1.0)), 10: ('17Networks_LH_VisCent_ExStr_9', (0.470588, 0.0705882, 0.552941, 1.0)), 11: ('17Networks_LH_VisCent_ExStr_10', (0.470588, 0.0705882, 0.556863, 1.0)), 12: ('17Networks_LH_VisCent_ExStr_11', (0.470588, 0.0745098, 0.529412, 1.0)), 13: ('17Networks_LH_VisPeri_ExStrInf_1', (1.0, 

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [32]:
import pickle
import nibabel as nib
import numpy as np
import os

# Parameters
add_bigbrain_data = True
do_parcellation = False
baseDir = '/Users/dennis.jungchildmind.org/Downloads/exvivo/'
baseDir_bigbrain = '/Users/dennis.jungchildmind.org/Downloads/BigBrain/thickness/resample/'

# Atlas setup
atlas_path = '/Users/dennis.jungchildmind.org/OneDrive - Child Mind Institute/parcellation/schaefer2018/Schaefer2018_400Parcels_17Networks_order.dlabel.nii'
atlas = nib.load(atlas_path).get_fdata()[0].astype(int)
atlas_data_lh = atlas[:len(atlas)//2]
atlas_data_rh = atlas[len(atlas)//2:]
atlas_data_rh = atlas_data_rh - np.min(atlas_data_rh[atlas_data_rh != 0]) + 1
atlas_data_rh[atlas_data_rh <= 0] = 0

# Get all subject directories in the baseDir
subject_dirs = [d for d in os.listdir(baseDir) if os.path.isdir(os.path.join(baseDir, d))]
l54_idx = next(i for i, s in enumerate(subject_dirs) if 'I54' in s)
print(f"Index of L54: {l54_idx}")
subject_dirs.insert(0, subject_dirs.pop(l54_idx))

data_types = ['infra', 'supra', 'relative', 'ratio_supra', 'ratio_infra', 'total', 'diff']
hemispheres = {'lh': 'left', 'rh': 'right'}
data = {hemi: {dtype: [] for dtype in data_types} for hemi in hemispheres}

def clean_data(data_array):
    masked_array = np.ma.masked_invalid(data_array)
    cleaned_array = np.array(masked_array.filled(0))
    return cleaned_array

def load_thickness_exvivo_data(base_dir, subject_dir, hemi, suffix):
    return {
        'infra': nib.load(f'{base_dir}{subject_dir}/{hemi}.thickness.wm.inf.{suffix}.shape.gii').darrays[0].data,
        'supra': nib.load(f'{base_dir}{subject_dir}/{hemi}.thickness.inf.pial.{suffix}.shape.gii').darrays[0].data,
        'total': nib.load(f'{base_dir}{subject_dir}/{hemi}.thickness.{suffix}.shape.gii').darrays[0].data
    }

def load_thickness_bigbrain_data(base_dir, hemi):
    return {
        'infra': nib.load(f'{base_dir}/{hemi}.3-6.32k.shape.gii').darrays[0].data,
        'supra': nib.load(f'{base_dir}/{hemi}.0-3.32k.shape.gii').darrays[0].data,
        'total': nib.load(f'{base_dir}/{hemi}.0-6.32k.shape.gii').darrays[0].data
    }

def calculate_derived_measurements(thickness_data):
    return {
        'relative': clean_data(np.divide(thickness_data['supra'], thickness_data['infra'], out=np.zeros_like(thickness_data['supra']), where=thickness_data['infra'] != 0)),
        'ratio_supra': clean_data(np.divide(thickness_data['supra'], thickness_data['total'], out=np.zeros_like(thickness_data['supra']), where=thickness_data['total'] != 0)),
        'ratio_infra': clean_data(np.divide(thickness_data['infra'], thickness_data['total'], out=np.zeros_like(thickness_data['infra']), where=thickness_data['total'] != 0)),
        'diff': clean_data((thickness_data['infra'] - thickness_data['supra']) / (thickness_data['infra'] + thickness_data['supra']))
    }

lh_subjects_name = []
rh_subjects_name = []

for subjectDir in subject_dirs:
    for hemi in hemispheres:
        surf_file = f'{baseDir}{subjectDir}/{hemi}.pial.32k_fs_LR.surf.gii'
        if os.path.exists(surf_file):
            if hemi == 'lh':
                lh_subjects_name.append(subjectDir.replace('_new_confidence', ''))
            else:
                rh_subjects_name.append(subjectDir.replace('_new_confidence', ''))
            suffix = '32k_fs_LR'
            thickness_data = load_thickness_exvivo_data(baseDir, subjectDir, hemi, suffix)
            thickness_data.update(calculate_derived_measurements(thickness_data))
            for dtype in data_types:
                reshaped = thickness_data[dtype].reshape(-1, 1)
                data[hemi][dtype] = np.concatenate((data[hemi][dtype], reshaped), axis=1) if len(data[hemi][dtype]) > 0 else reshaped

if add_bigbrain_data:
    print('?')
    for hemi in hemispheres:
        thickness_data = load_thickness_bigbrain_data(baseDir_bigbrain, hemi)
        thickness_data.update(calculate_derived_measurements(thickness_data))
        for dtype in data_types:
            reshaped = thickness_data[dtype].reshape(-1, 1)
            data[hemi][dtype] = np.concatenate((data[hemi][dtype], reshaped), axis=1) if len(data[hemi][dtype]) > 0 else reshaped
    lh_subjects_name.append('bigbrain')
    rh_subjects_name.append('bigbrain')

if do_parcellation:
    def parcellate_data(data_array, atlas_data):
        # Placeholder: actual parcellation function should be defined elsewhere
        raise NotImplementedError("Parcellation function not implemented.")
    for hemi in hemispheres:
        atlas_data = atlas_data_lh if hemi == 'lh' else atlas_data_rh
        for dtype in data_types:
            tmp_data = []
            for i in range(data[hemi][dtype].shape[1]):
                parcellated = parcellate_data(data[hemi][dtype][:, i], atlas_data)
                tmp_data.append(parcellated)
            data[hemi][dtype] = np.array(tmp_data).T







# Calculate correlation coefficient w.r.t bigbrain
thickness_corr_wrt_bigbrain = {}
for layer in ['supra', 'infra','total']:
    for hemi in ['lh', 'rh']:
        if hemi not in thickness_corr_wrt_bigbrain:
            thickness_corr_wrt_bigbrain[hemi] = {}

        if hemi == 'lh':
            mask = atlas_data_lh > 0
        else:
            mask = atlas_data_rh > 0

        corr_matrix = np.corrcoef(data[hemi][layer][mask,:].T)
        thickness_corr_wrt_bigbrain[hemi][layer] = corr_matrix[-1, :]

subject_names = {'lh': lh_subjects_name, 'rh': rh_subjects_name}

with open('thickness_corr_wrt_bigbrain_32k.pkl', 'wb') as f:
    pickle.dump({'subject_names': subject_names, 'thickness_corr_wrt_bigbrain': thickness_corr_wrt_bigbrain}, f)


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/var/folders/71/4g31r9017kq32wt_mx1khjdm0000gn/T/ipykernel_84555/3483506978.py:54: RuntimeWarning: invalid value encountered in divide
  'diff': clean_data((thickness_data['infra'] - thickness_data['supra']) / (thickness_data['infra'] + thickness_data['supra']))


Index of L54: 6
?


In [36]:
print(thickness_corr_wrt_bigbrain['lh']['total'])

[0.20972295 0.22389229 0.22753085 0.20184256 0.17971734 0.18829168
 0.2171324  0.25804937 0.20496636 0.2038515  0.16064065 1.        ]


In [5]:

# Calculate correlation coefficient w.r.t bigbrain
thickness_corr_wrt_bigbrain = {}
for layer in ['supra', 'infra']:
    for hemi in ['lh', 'rh']:
        if hemi not in thickness_corr_wrt_bigbrain:
            thickness_corr_wrt_bigbrain[hemi] = {}

        if hemi == 'lh':
            mask = atlas_data_lh > 0
        else:
            mask = atlas_data_rh > 0

        corr_matrix = np.corrcoef(data[hemi][layer][mask,:].T)
        thickness_corr_wrt_bigbrain[hemi][layer] = corr_matrix[-1, :]

subject_names = {'lh': lh_subjects_name, 'rh': rh_subjects_name}

with open('thickness_corr_wrt_bigbrain_32k.pkl', 'wb') as f:
    pickle.dump({'subject_names': subject_names, 'thickness_corr_wrt_bigbrain': thickness_corr_wrt_bigbrain}, f)


In [38]:
print(data[hemi][layer][mask,-1].shape)

(29308,)


In [44]:
from sklearn.metrics import mutual_info_score
thickness_mi_wrt_bigbrain = {}
for layer in ['supra', 'infra','total']:
    for hemi in ['lh', 'rh']:
        if hemi not in thickness_mi_wrt_bigbrain:
            thickness_mi_wrt_bigbrain[hemi] = {}
        tmp_mi = []
        
        if hemi == 'lh':
            mask = atlas_data_lh > 0
        else:
            mask = atlas_data_rh > 0

        # Discretize the continuous data before computing mutual_info_score to avoid the warning.
        # We'll use numpy's digitize to bin the data into, e.g., 10 bins.
        n_bins = 100
        x = data[hemi][layer][mask, :]
        # Discretize each column
        x_binned = np.zeros_like(x, dtype=int)
        for col in range(x.shape[1]):
            bins = np.histogram_bin_edges(x[:, col], bins=n_bins)
            x_binned[:, col] = np.digitize(x[:, col], bins) - 1  # bins start at 1
        for i in range(x_binned.shape[1]):
            tmp_mi.append(mutual_info_score(x_binned[:, i], x_binned[:, -1]))
        thickness_mi_wrt_bigbrain[hemi][layer] = tmp_mi
   
   
    
with open('thickness_mi_wrt_bigbrain_32k.pkl', 'wb') as f:
    pickle.dump({'subject_names': subject_names, 'thickness_mi_wrt_bigbrain': thickness_mi_wrt_bigbrain}, f)

    

In [42]:
print(thickness_mi_wrt_bigbrain['lh']['total'])

[0.12233188435683723, 0.12330782769029423, 0.13060170541084476, 0.10734727297448982, 0.1247288685292772, 0.11491031399341722, 0.11394535408807402, 0.11849995604844896, 0.12470718455228656, 0.13538623308199207, 0.11793843130378054, 3.8166909361818346]
